<a href="https://colab.research.google.com/github/danielbauer1860/LDS_Project/blob/main/fine_tuning/Llama_2_fine_tuning_news_8epochs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 tensorboard huggingface_hub[cli] xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [15]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
import pandas as pd
import os
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    )
from datasets import load_dataset, Dataset
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model
import transformers

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Linguistic Data Science/data/bnc_baby_texts.csv', sep='|')
df

,text,category
0,"Why do certain images matter to one, and why i...",ACA
1,"In many developing countries, the immunogenici...",ACA
2,Anthony Kenny The British Library 1992 Scholar...,ACA
3,Beforehand he worked as a probation officer fo...,ACA
4,CONTINUITY AND CHANGE are features of every po...,ACA
...,...,...
177,"Artworks THE Friends of Oriel Ynys Mon, Llange...",NEWS
178,THERE will be those who portray the report of ...,NEWS
179,BUCKINGHAM PALACE 30 September: The Princess R...,NEWS
180,The Scottish Landowners' Federation unequivoca...,NEWS


In [6]:
df_aca, df_dem, df_fic, df_news = [y for x, y in df.groupby(['category'])]

<ipython-input-6-db6973ab6e89>:1: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  df_aca, df_dem, df_fic, df_news = [y for x, y in df.groupby(['category'])]


In [7]:
df_aca

,text,category
0,"Why do certain images matter to one, and why i...",ACA
1,"In many developing countries, the immunogenici...",ACA
2,Anthony Kenny The British Library 1992 Scholar...,ACA
3,Beforehand he worked as a probation officer fo...,ACA
4,CONTINUITY AND CHANGE are features of every po...,ACA
5,Records can be retrieved directly from files t...,ACA
6,The party was in a mutinous condition in Novem...,ACA
7,One of the chief peculiarities of this treatis...,ACA
8,"Mitchel Langford, David J. Maguire and David J...",ACA
9,Crime and criminals are topics of endless fasc...,ACA


In [8]:
aca_dataset = Dataset.from_pandas(df_aca)
fic_dataset = Dataset.from_pandas(df_fic)
news_dataset = Dataset.from_pandas(df_news)

In [9]:
aca_dataset

Dataset({
    features: ['text', 'category', '__index_level_0__'],
    num_rows: 30
})

# Fine-tuning Llama 2

Norouzi (2023) describes an efficient way to handle Llama 2 despite the system RAM limitations that Google Colab comes with and was, therefore, used as the main source throughout this section. For more details see: [Mastering Llama 2: A Comprehensive Guide to Fine-Tuning in Google Colab](https://artificialcorner.com/mastering-llama-2-a-comprehensive-guide-to-fine-tuning-in-google-colab-bedfcc692b7f). Additionally, [this notebook](https://colab.research.google.com/drive/12dVqXZMIVxGI0uutU6HG9RWbWPXL3vts?authuser=2#scrollTo=qmA4G6C64dJ4) was used as a reference for further parameters.

In [10]:
# Declaring the model name; as described in the paper, the 7 billion parameter version of Llama 2 is used
model_name = "meta-llama/Llama-2-7b-hf"

The parameters for the quantization config need to be defined:

In [11]:
# Load the entire model on the GPU 0
device_map = {"": 0}

# Set base model loading in 4-bits
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = torch.float16

# Quantization type (fp4 or nf4); nf4 is shown to be better in the QLoRA paper
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [16]:
# Initialize the quantization config using the previously declared parameters
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant
)

Now that we got both the `device_map` set to one GPU and the quantization config `bnb_config` initialized, the base model can be loaded into the system without any memory issues:

In [17]:
# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
    quantization_config=bnb_config,
)
model.config.use_cache = False

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [18]:
# Load the corresponding tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

As the original tokenizer has `pad_id` set to -1, we need to to define a custom padding token. While the [Llama 2 huggingface documentation](https://huggingface.co/docs/transformers/main/model_doc/llama2#overview) recommends using `tokenizer.add_special_tokens({"pad_token":"<pad>"})`, Norouzi points out that this can introduce CUDA-related errors. Therefore, he sees directly setting the `pad_token` as a safer option.




In [19]:
#Define a custom padding token
tokenizer.pad_token = tokenizer.eos_token

# Set the padding direction to the right
tokenizer.padding_side = "right"

These parameters are set in accordance to the QLoRA method.

In [20]:
# LoRA attention dimension
lora_r = 64
# Alpha for LoRA scaling
lora_alpha = 16
# Dropout probability for LoRA
lora_dropout = 0.1

In [21]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [22]:
output_dir = "/content/drive/MyDrive/results/news"
final_checkpoint_dir = os.path.join(output_dir, "8e_final_checkpoint")

While most hyperparameters remain the same throughout the different variants, the number of epochs will be varied from 2 to 4 and 8.

In [23]:
per_device_train_batch_size = 1
gradient_accumulation_steps = 1
num_train_epochs = 8 #2, 4
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 5
learning_rate = 4e-5
max_grad_norm = 0.3
warmup_ratio = 0.03
lr_scheduler_type = "constant"

In [24]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    num_train_epochs=num_train_epochs
)

In [25]:
max_seq_length = 2500
packing = False

In [26]:
news_trainer = SFTTrainer(
    model=model,
    train_dataset=news_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/97 [00:00<?, ? examples/s]

In [27]:
resume_checkpoint = False

In [28]:
transformers.logging.set_verbosity_info()

In [29]:
for name, module in news_trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [30]:
news_trainer.train(resume_checkpoint)

***** Running training *****
  Num examples = 97
  Num Epochs = 8
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 776
  Number of trainable parameters = 33,554,432
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
5,2.170800
10,2.109100
15,2.221700
20,2.202700
25,2.352700
30,2.357100
35,2.171800
40,2.115700
45,2.445600
50,2.235000


Saving model checkpoint to /content/drive/MyDrive/results/news/checkpoint-100
tokenizer config file saved in /content/drive/MyDrive/results/news/checkpoint-100/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/results/news/checkpoint-100/special_tokens_map.json
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Saving model checkpoint to /content/drive/MyDrive/results/news/checkpoint-200
tokenizer config file saved in /content/drive/MyDrive/results/news/checkpoint-200/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/results/news/checkpoint-2

TrainOutput(global_step=776, training_loss=2.1405299262902173, metrics={'train_runtime': 7106.2365, 'train_samples_per_second': 0.109, 'train_steps_per_second': 0.109, 'total_flos': 3.390623158173696e+16, 'train_loss': 2.1405299262902173, 'epoch': 8.0})

Saving and uploading the model:

In [31]:
model_to_save = news_trainer.model.module if hasattr(news_trainer.model, 'module') else news_trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [32]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

In [33]:
model.push_to_hub(f"dbauer1860/llama-2-bnc-baby-news-{num_train_epochs}-epochs", create_pr=1)

Uploading the following files to dbauer1860/llama-2-bnc-baby-news-8-epochs: adapter_model.bin,adapter_config.json,README.md


CommitInfo(commit_url='https://huggingface.co/dbauer1860/llama-2-bnc-baby-news-8-epochs/commit/b287c4c25ea87fdfb0f510211376babf8f1ef94c', commit_message='Upload model', commit_description='', oid='b287c4c25ea87fdfb0f510211376babf8f1ef94c', pr_url='https://huggingface.co/dbauer1860/llama-2-bnc-baby-news-8-epochs/discussions/2', pr_revision='refs/pr/2', pr_num=2)

Exporting the relevant log information to plot in another notebook

In [34]:
news_trainer.state.log_history

[{'loss': 2.1708, 'learning_rate': 4e-05, 'epoch': 0.05, 'step': 5},
 {'loss': 2.1091, 'learning_rate': 4e-05, 'epoch': 0.1, 'step': 10},
 {'loss': 2.2217, 'learning_rate': 4e-05, 'epoch': 0.15, 'step': 15},
 {'loss': 2.2027, 'learning_rate': 4e-05, 'epoch': 0.21, 'step': 20},
 {'loss': 2.3527, 'learning_rate': 4e-05, 'epoch': 0.26, 'step': 25},
 {'loss': 2.3571, 'learning_rate': 4e-05, 'epoch': 0.31, 'step': 30},
 {'loss': 2.1718, 'learning_rate': 4e-05, 'epoch': 0.36, 'step': 35},
 {'loss': 2.1157, 'learning_rate': 4e-05, 'epoch': 0.41, 'step': 40},
 {'loss': 2.4456, 'learning_rate': 4e-05, 'epoch': 0.46, 'step': 45},
 {'loss': 2.235, 'learning_rate': 4e-05, 'epoch': 0.52, 'step': 50},
 {'loss': 2.2797, 'learning_rate': 4e-05, 'epoch': 0.57, 'step': 55},
 {'loss': 2.2179, 'learning_rate': 4e-05, 'epoch': 0.62, 'step': 60},
 {'loss': 2.0966, 'learning_rate': 4e-05, 'epoch': 0.67, 'step': 65},
 {'loss': 2.404, 'learning_rate': 4e-05, 'epoch': 0.72, 'step': 70},
 {'loss': 2.2953, 'learn

In [41]:
loss_values = []
epochs = []

for step in news_trainer.state.log_history:
  for k, v in step.items():
    if k in ['loss', 'train_loss']:
      loss_values.append(v)
    if k == 'epoch'and v not in epochs:
      epochs.append(v)

len(loss_values)

156

In [42]:
df = pd.DataFrame({'loss': loss_values, 'epoch': epochs})
df

,loss,epoch
0,2.17080,0.05
1,2.10910,0.10
2,2.22170,0.15
3,2.20270,0.21
4,2.35270,0.26
...,...,...
151,1.89320,7.84
152,2.02090,7.89
153,2.06650,7.94
154,1.92870,7.99


In [43]:
df.to_csv("/content/drive/MyDrive/Linguistic Data Science/data/logs/news_8e_logs.csv", sep=";", index=False)